# API caller

Notebook used to test the API `be-volumetry`

In [4]:
# Install required packages
!pip install --quiet requests
!pip install --quiet requests-toolbelt

You should consider upgrading via the '/home/zimmy/.cache/pypoetry/virtualenvs/be-centratura-6766qhX0-py3.7/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/zimmy/.cache/pypoetry/virtualenvs/be-centratura-6766qhX0-py3.7/bin/python -m pip install --upgrade pip' command.


In [5]:
# Config
HOST = "https://cloudrun-be-volumetry-h4dupqlq3q-ew.a.run.app"
HEALTH_API = "/api/healthz"
VOLUMETRY_API = "/api/analysis/volumetry"
IDENTITY_TOKEN = !gcloud auth print-identity-token
IDENTITY_TOKEN = IDENTITY_TOKEN[0]

### Check cloud run connection

In [6]:
import requests

headers = {'Content-Type':'application/json',
            'Authorization': f'Bearer {IDENTITY_TOKEN}'}

r = requests.get(f"{HOST}{HEALTH_API}",  headers=headers)

assert r.status_code == 200, f"Cannot connect to clud run at {HOST}, status:{r.status_code} - error:{r.text}"
print(f"Connection to cloud run established!")

Connection to cloud run established!


### BE-volumetry API

In [ ]:
# Download the testing images
!gsutil cp -r gs://mlspec-volumetry/assets .

Copying gs://mlspec-volumetry/assets/3200/CX/center.jpg...
Copying gs://mlspec-volumetry/assets/3200/DX/20200625131458530.jpg...           
/ [2 files][  1.2 MiB/  1.2 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://mlspec-volumetry/assets/3200/SX/20200625131458529.jpg...


In [ ]:
import uuid
from os.path import basename

correlation_id = uuid.uuid4().hex
cam_left_path = "./assets/3200/CX/center.jpg"
cam_center_path = "./assets/3200/DX/20200625131458530.jpg"
cam_right_path = "./assets/3200/SX/20200625131458529.jpg"

In [ ]:
import requests
from pprint import pprint
from requests_toolbelt import MultipartEncoder


data = MultipartEncoder(
        fields={'correlationId': correlation_id,
                'camLeft': (basename(cam_left_path), open(cam_left_path, 'rb'), 'image/jpeg'),
                'camCenter': (basename(cam_center_path), open(cam_center_path, 'rb'), 'image/jpeg'),
                'camRight': (basename(cam_right_path), open(cam_right_path, 'rb'), 'image/jpeg')}
        )

r = requests.post(f'{HOST}{VOLUMETRY_API}', data=data,
                  headers={'Content-Type':  data.content_type,
                          'Authorization': f'Bearer {IDENTITY_TOKEN}'})

print("API response:")
pprint(r.json())